In [1]:
import numpy as np

In [2]:
one_d = np.load('../arrays/labels.npy')

In [3]:
np.sum(one_d)

219216.0

In [4]:
len(one_d)

129741886

In [5]:
np.sum(one_d)/len(one_d)

0.0016896316737680227

In [6]:
frac = one_d[int(len(one_d)*0):int(len(one_d)*0.01)]
np.sum(frac)/len(frac)

0.00070447612103423872

In [7]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import numpy as np
import sys

import sklearn.metrics as met

import keras

from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout


class Cust_metrics(keras.callbacks.Callback):
	def on_train_begin(self, logs={}):
		self.aucs = []
		#self.losses = []

	def on_train_end(self, logs={}):
		return

	def on_epoch_begin(self, epoch, logs={}):
		return

	def on_epoch_end(self, epoch, logs={}):

		epoch_pred = self.model.predict(coord_test)
		epoch_bin_pred = np.array(epoch_pred > 0.5).astype(int)
		#self.losses.append(logs.get('loss'))

		#self.aucs.append(roc_auc_score(labels_test, y_pred))
		print('\n', 'Custom epoch metrics: ', 'auc: ', met.roc_auc_score(labels_test, epoch_pred), end=' - ')

		print('mcc: ', met.matthews_corrcoef(labels_test,epoch_bin_pred))

		return

	def on_batch_begin(self, batch, logs={}):
		return

	def on_batch_end(self, batch, logs={}):
		return


def prepare_data(labels_path, coord_path):

	# Load coordinate and labels data
	labels = np.load(labels_path)
	coord = np.load(coord_path)
	#names = np.load('arrays/names.npy')

	# Preprocess coordinate data (zero centered and between -1 and 1)
	#coord -= 0.5
	#coord *= 2

	return labels, coord


def subset_data(labels, coord, keep=1.0, train_split=1.0):

	#Subsetting data
	labels = labels[0:int(keep*len(labels))]
	coord = coord[0:int(keep*len(coord))]

	# Splitting data
	labels_train = labels[0:int(len(labels)*train_split)]
	coord_train = coord[0:int(len(coord)*train_split)]
	labels_test = labels[int(len(labels)*train_split):]
	coord_test = coord[int(len(coord)*train_split):]

	return labels_train, labels_test, coord_train, coord_test


def build_model(indim, reg, drop, nlayers, nunits):

	# Setting up model
	model = Sequential()

	model.add(Dense(nunits, input_dim=indim, activation='relu', kernel_regularizer=regularizers.l2(reg), kernel_initializer='he_normal'))
	model.add(Dropout(drop))

	for i in range(nlayers):
		model.add(Dense(nunits, activation='relu', kernel_regularizer=regularizers.l2(reg), kernel_initializer='he_normal'))
		model.add(Dropout(drop))

	model.add(Dense(1, activation='sigmoid'))

	return model


def compile_mod(model):

	# Compiling the model
	adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
	#sdg = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=True)

	model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

	return model

def set_callbacks():

	cb = []

	cb.append(keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0))
	#cb.append(keras.callbacks.LearningRateScheduler(schedule))
	cb.append(keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.00001, patience=10, verbose=1, mode='auto'))
	cb.append(Cust_metrics())

	return cb

In [8]:
print("----Loading and processing data----")
labels, coord = prepare_data('../arrays/labels.npy', '../arrays/coord_all_interactions.npy')
labels_train, labels_test, coord_train, coord_test = subset_data(labels, coord, keep=1, train_split=0.75)

----Loading and processing data----


In [9]:
from imblearn.over_sampling import SMOTE
from time import time
sm = SMOTE(random_state=42, n_jobs=10)
print("SM ready")
t1 = time()
coord_075_resampled, labels_075_resampled = sm.fit_sample(coord_train, labels_train)
t2 = time()
print("Time for resampling:", t2-t1)

SM ready
Time for resampling: 727.2790672779083


In [10]:
np.save('../arrays/4D_075_coord_resampled.npy',coord_075_resampled)
np.save('../arrays/labels_075_resampled.npy',labels_075_resampled)

In [11]:
np.save('../arrays/4D_025_coord.npy',coord_test)
np.save('../arrays/labels_025.npy',labels_test)

In [28]:
coord_075_resampled.shape

(194296950, 8)

In [2]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import numpy as np
import sys
import pickle

import sklearn.metrics as met

import keras

from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout


class Cust_metrics(keras.callbacks.Callback):
	def on_train_begin(self, logs={}):
		self.aucs = []
		#self.losses = []

	def on_train_end(self, logs={}):
		return

	def on_epoch_begin(self, epoch, logs={}):
		return

	def on_epoch_end(self, epoch, logs={}):

		epoch_pred = self.model.predict(coord_test)
		epoch_bin_pred = np.array(epoch_pred > 0.5).astype(int)
		#self.losses.append(logs.get('loss'))

		#self.aucs.append(roc_auc_score(labels_test, y_pred))
		print('\n', 'Custom epoch metrics:  auc:', met.roc_auc_score(labels_test, epoch_pred), end=' - ')

		print('mcc:', met.matthews_corrcoef(labels_test,epoch_bin_pred), ' - acc:', met.accuracy_score(labels_test,epoch_bin_pred), end=' - ')

		print("Prediciton  MAX:", max(epoch_pred) , "Prediction MIN:", min(epoch_pred), end=' - ')

		print("Fraction of positive predictions:", np.sum(epoch_bin_pred)/len(epoch_bin_pred))

		return

	def on_batch_begin(self, batch, logs={}):
		return

	def on_batch_end(self, batch, logs={}):
		return


def normalise_coord(array_list):

	for i in range(len(array_list)):
		array_list[i] -= 0.5
		array_list[i] *= 2

	return array_list


def subset_data(labels, coord, keep=1.0, train_split=1.0):

	#Subsetting data
	labels = labels[0:int(keep*len(labels))]
	coord = coord[0:int(keep*len(coord))]

	# Splitting data
	labels_train = labels[0:int(len(labels)*train_split)]
	coord_train = coord[0:int(len(coord)*train_split)]
	labels_test = labels[int(len(labels)*train_split):]
	coord_test = coord[int(len(coord)*train_split):]

	return labels_train, labels_test, coord_train, coord_test


def build_model(indim, reg, drop, nlayers, nunits, act):

	# Setting up model
	model = Sequential()

	model.add(Dense(nunits, input_dim=indim, activation=act, kernel_regularizer=regularizers.l2(reg), kernel_initializer='he_normal'))
	model.add(Dropout(drop))

	for i in range(nlayers):
		model.add(Dense(nunits, activation=act, kernel_regularizer=regularizers.l2(reg), kernel_initializer='he_normal'))
		model.add(Dropout(drop))

	model.add(Dense(1, activation='sigmoid'))

	return model


def compile_mod(model):

	# Compiling the model
	adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
	#sdg = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=True)

	model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

	return model

def set_callbacks():

	cb = []

	cb.append(keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0))
	#cb.append(keras.callbacks.LearningRateScheduler(schedule))
	cb.append(keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.00001, patience=10, verbose=1, mode='auto'))
	cb.append(Cust_metrics())

	return cb


############################### Main Program ###################################

# Setting hyperparameters
reg = 0.0
drop = 0.0
nlayers = 1
nunits = 128


print("----Loading Data----")
labels_train = np.load('../arrays/labels_075_resampled.npy')
labels_test = np.load('../arrays/labels_025.npy')
coord_train = np.load('../arrays/4D_075_cint_resampled_8x194m.npy')
coord_test = np.load('../arrays/4D_025_cint_8x32m.npy')


print("----Processing Data----")
coord_test, coord_train = normalise_coord([coord_test, coord_train])
#labels_train, labels_test, coord_train, coord_test = subset_data(labels, coord, keep=1, train_split=0.75)


print("----Setting up the model----")
model = build_model(coord_train.shape[1], reg, drop, nlayers, nunits, 'relu')


print("----Compiling the model----")
compiled_mod = compile_mod(model)


print("----Model training----")
compiled_mod.fit(coord_train, labels_train, epochs=1, batch_size=64, callbacks=set_callbacks())#,validation_split=0.25)


pred = compiled_mod.predict(coord_test,batch_size=64, verbose=0)
#print('\n','Final Prediciton Score Summary (Validation Set Size:', np.abs(len(pred)),'):' , '\n', "MAX:", max(pred),'\n' , "MIN:", min(pred),'\n')

print(compiled_mod.get_weights())
#pickle.dump(weights,open( "weights.p", "wb"))
# Evaluate model with test data
#score = model.evaluate(coord_test, coord_test, batch_size=128)
#print(score)


----Loading Data----
----Processing Data----
----Setting up the model----
----Compiling the model----
----Model training----
Epoch 1/1
194296064/194296950 [============================>.] - ETA: 0s - loss: 0.4447 - acc: 0.7868
 Custom epoch metrics:  auc: 0.78026529663 - mcc: 0.0417260783928  - acc: 0.752192753662 - Prediciton  MAX: [ 1.] Prediction MIN: [  3.39155864e-32] - Fraction of positive predictions: 0.248422529507
194296950/194296950 [==============================] - 3617s - loss: 0.4447 - acc: 0.7868  
[array([[ -5.67328691e-01,  -2.16078186e+00,  -1.93456970e-02, ...,
          2.00844854e-02,   2.76651096e+00,   4.40695047e-01],
       [  2.61440945e+00,   2.02521876e-01,   3.01957899e-03, ...,
          9.59731415e-02,   1.34519506e+00,  -4.20635134e-01],
       [  1.13600039e+00,  -2.14128971e-01,  -9.70843062e-02, ...,
         -5.05073890e-02,   5.83019733e-01,  -2.16224477e-01],
       ..., 
       [ -1.22470826e-01,  -6.47851050e-01,   4.73687798e-01, ...,
          

In [37]:
bin_pred = np.array(pred > 0.8).astype(int)
print("Fraction of positive predictions:", np.sum(bin_pred)/len(bin_pred))

Fraction of positive predictions: 0.0597810631521


In [38]:
met.matthews_corrcoef(labels_test,bin_pred)

0.049717641315649735